In [1]:
client_id = ''
client_secret = ''

import pandas as pd
from datetime import datetime, timedelta
import requests
import re
def handle_authorization(data):
    
    access_token = data["access_token"]
    print(f" {access_token}")
    return access_token
    
def get_twitch_authorization(client_id, client_secret):
    url = f"https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
    }
    
    response = requests.post(url, params=params)
    data = response.json()
    
    handle_authorization(data)



# Obtén la autorización de Twitch
get_twitch_authorization(client_id, client_secret)

 9lunj5lzu8sm85vw8ezlu4wlb35pu5


In [2]:
oauth_token = ''

In [3]:
def extract_channel_name(url):
    pattern = r"https?:\/\/(?:www\.|go\.)?twitch\.tv\/([a-zA-Z0-9_]{4,25})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None
def get_streamer_data(client_id, url):
    channel_name = extract_channel_name(url)
    if not channel_name:
        return None
    
    url = f"https://api.twitch.tv/helix/users"
    
    params = {
        "login": channel_name
    }
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    print(data)
    if "data" in data and len(data["data"]) > 0:
        user_id = data["data"][0]["id"]
        stream_url = f"https://api.twitch.tv/helix/streams?user_id={user_id}"
        response = requests.get(stream_url, headers=headers)
        stream_data = response.json()
        
        if "data" in stream_data and len(stream_data["data"]) > 0:
            return stream_data["data"][0]
    
    return None
url = "https://www.twitch.tv/ikercasillas"
streamer_data = get_streamer_data(client_id, url)

if streamer_data:
    data = f"Datos del streamer {channel_url}:"

{'data': [{'id': '847518400', 'login': 'ikercasillas', 'display_name': 'ikercasillas', 'type': '', 'broadcaster_type': 'partner', 'description': 'Alguna vez volaba... ', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/f51737c8-12a3-4790-8e81-3c8058162ca9-profile_image-300x300.png', 'offline_image_url': '', 'view_count': 0, 'created_at': '2022-11-14T16:32:41Z'}]}


In [4]:
data = {'data': [{'id': '847518400', 'login': 'ikercasillas', 'display_name': 'ikercasillas', 'type': '', 'broadcaster_type': 'partner', 'description': 'Alguna vez volaba... ', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/f51737c8-12a3-4790-8e81-3c8058162ca9-profile_image-300x300.png', 'offline_image_url': '', 'view_count': 0, 'created_at': '2022-11-14T16:32:41Z'}]}
data_dict = data['data'][0]
data_dict
id_canal =  list(data_dict.values())[0]
id_canal

'847518400'

In [5]:
def obtener_vods(id_canal):
    # Calcular la fecha hace un año desde hoy
    fecha_inicio = datetime.now() - timedelta(days=730)
    fecha_inicio_str = fecha_inicio.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100"

    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }

    vods_data = []

    while url:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            vods = data["data"]

            for vod in vods:
                vod_id = int(vod["id"])
                fecha_inicio = datetime.strptime(vod["created_at"], "%Y-%m-%dT%H:%M:%SZ")
                fecha_final = datetime.strptime(vod["published_at"], "%Y-%m-%dT%H:%M:%SZ")
                duracion = vod["duration"]
                visitas = int(vod["view_count"])
                nombre = vod["title"].lower().replace(' ', '_')
                descripcion = vod["description"]
                url_vod = f"https://www.twitch.tv/videos/{vod_id}"

                vods_data.append({
                    "ID del VOD": vod_id,
                    "URL del VOD": url_vod,
                    "Nombre del VOD": nombre,
                    "Duración": duracion,
                    "Fecha de inicio": fecha_inicio,
                    "Fecha de finalización": fecha_final,
                    "Visitas": visitas,
                    
                })

            pagination = data.get("pagination", None)
            if pagination and "cursor" in pagination:
                cursor = pagination["cursor"]
                url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100&after={cursor}"
            else:
                url = None

        else:
            print("Se produjo un error al realizar la solicitud GET.")
            return None

    # Crear un DataFrame a partir de los datos
    df_vods = pd.DataFrame(vods_data)

    return df_vods

df = obtener_vods(id_canal)
df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID del VOD             23 non-null     int64         
 1   URL del VOD            23 non-null     object        
 2   Nombre del VOD         23 non-null     object        
 3   Duración               23 non-null     object        
 4   Fecha de inicio        23 non-null     datetime64[ns]
 5   Fecha de finalización  23 non-null     datetime64[ns]
 6   Visitas                23 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 7.1 KB


,ID del VOD,URL del VOD,Nombre del VOD,Duración,Fecha de inicio,Fecha de finalización,Visitas
0,1882950509,https://www.twitch.tv/videos/1882950509,último_programa_|_pov_+_ojo_del_santo_|_#kings...,1h9m21s,2023-07-27 11:00:20,2023-07-27 11:00:20,10013
1,1881381329,https://www.twitch.tv/videos/1881381329,matchday_|_1k_f.c._vs_aniquiladores_|_#kingsle...,1h25m16s,2023-07-25 16:39:25,2023-07-25 16:39:25,60638
2,1880338993,https://www.twitch.tv/videos/1880338993,extratime_+_ojo_del_santo_|_1k_f.c._|_#kingsle...,1h4m57s,2023-07-24 11:31:16,2023-07-24 11:31:16,8244
3,1878697102,https://www.twitch.tv/videos/1878697102,matchday_|_1k_f.c._vs_ultimate_mostoles_|_#kin...,1h52m32s,2023-07-22 13:33:14,2023-07-22 13:33:14,53074
4,1876823807,https://www.twitch.tv/videos/1876823807,pov_+_ojo_del_santo_|_#kingsleaguej11_|,1h4m5s,2023-07-20 11:01:06,2023-07-20 11:01:06,7609


In [6]:
df.to_csv('twitch_api_ikercasillas_csv.csv', index=False)